# 구글맵 크롤링
- selenium과 beautifulsoup을 이용한 구글맵 크롤링 코드

In [ ]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from time import sleep
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
import re
from selenium.webdriver.common.keys import Keys

In [ ]:
import os

#현재 폴더 경로; 작업 폴더 기준
print(os.getcwd())

/Users/nang/Desktop/Projects/WCRC2023DataContest


In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(executable_path = './chromedriver', options=options)

/var/folders/dr/6msfb1qj7w9cvw_lz86pmzq40000gn/T/ipykernel_33127/918247877.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = './chromedriver', options=options)


In [ ]:
def crawl(num, lst):
    # num : 크롤링하고자 하는 리뷰 개수
    # scroll_num : 스크롤 횟수 - 1번 스크롤시 10개의 리뷰 더 추출 = num//10
    # lst : market list

    global market_name
    global star
    global review

    market_name, star, review = [], [], []

    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(executable_path = './chromedriver', options=options)

    for market in lst:
        if market == '수유시장':
            url = "https://www.google.co.kr/maps/place/%EC%88%98%EC%9C%A0%EC%8B%9C%EC%9E%A5(%EC%A3%BC)/data=!4m10!1m2!2m1!1z7IiY7Jyg7Iuc7J6l!3m6!1s0x357cbc6b9de2de6b:0x36014bd1c7c2a399!8m2!3d37.6308918!4d127.0232076!15sCgzsiJjsnKDsi5zsnqVaDyIN7IiY7JygIOyLnOyepZIBBm1hcmtldOABAA!16s%2Fg%2F1thq7twx?hl=ko&entry=ttu"

        elif market == "청량리청과물도매시장":
            url = "https://www.google.co.kr/maps/place/%EC%B2%AD%EB%9F%89%EB%A6%AC+%EC%B2%AD%EA%B3%BC%EB%AC%BC+%EB%8F%84%EB%A7%A4%EC%8B%9C%EC%9E%A5/data=!3m1!4b1!4m6!3m5!1s0x357cbb5793cc569f:0x3dcfeeb6dea8f95e!8m2!3d37.5798488!4d127.0433745!16s%2Fg%2F119vb5lz7?hl=ko&entry=ttu"

        elif market == "남대문시장":
            url = "https://www.google.co.kr/maps/place/%EB%82%A8%EB%8C%80%EB%AC%B8%EC%8B%9C%EC%9E%A5/data=!4m10!1m2!2m1!1z64Ko64yA66y47Iuc7J6l!3m6!1s0x357ca2f516e5887d:0xcd371b16a3ec2061!8m2!3d37.5591786!4d126.9776692!15sCg_rgqjrjIDrrLjsi5zsnqVaEiIQ64Ko64yA66y4IOyLnOyepZIBBm1hcmtldOABAA!16zL20vMGd3bnB3?hl=ko&entry=ttu"

        else:
            url = f"https://www.google.com/maps/search/{market}?entry=ttu"

        driver.get(url) ; sleep(10)  # GET URL

        try:
            driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)").click() # 리뷰

        except NoSuchElementException:
            driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div:nth-child(1) > div > a").click()
            sleep(1)
            driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div:nth-child(3) > div > div > button:nth-child(2)").click() # 리뷰

        sleep(5)
        driver.find_elements(By.CLASS_NAME, "DVeyrd")[2].click() ; sleep(3) # 정렬
        driver.find_element(By.CSS_SELECTOR, "#action-menu > div:nth-child(2)").click() ; sleep(3) # 최신순

        ### SCROLLING ###
        scroll = driver.find_element(By.CSS_SELECTOR, "#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf")
        for i in range(50):
            driver.execute_script("arguments[0].scrollBy(0,10000)", scroll)
            sleep(1)

        ### 더보기 ###
        MyEned = driver.find_elements(By.CLASS_NAME, "MyEned")

        for i in range(len(MyEned)):
            detail_button = MyEned[i].find_elements(By.TAG_NAME, 'span')
            if len(detail_button) >= 2:
                detail_button[1].click()
            else:
                continue

        ### PAGE SOURCE ###
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        for n in range(num):
            s = re.sub('[^0-9]','',soup.select("div.DU9Pgb > span:nth-child(1)")[n]['aria-label']) ### STAR ###
            r = soup.select("div.DU9Pgb + div")[n].text.lstrip() ### REVIEW ###

            market_name.append(market)
            star.append(s)
            review.append(r)

    driver.quit()

In [ ]:
market_lst = ['남대문시장',
              '수유시장',
              '청량리청과물도매시장'
              '망원시장',
              '동묘시장',
              '경동시장',
              '광장시장']

In [ ]:
crawl(300, market_lst)

/var/folders/dr/6msfb1qj7w9cvw_lz86pmzq40000gn/T/ipykernel_33127/109510741.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = './chromedriver', options=options)


In [ ]:
print(len(review))
print(len(market_name))
print(len(star))

900
900
900


In [ ]:
df = pd.DataFrame({'시장명': market_name,
              '별점': star,
             '리뷰내용': review})

In [ ]:
df.to_csv("googlemap_crawling.csv")

In [ ]:
df[df['리뷰내용']==''].groupby('시장명').count()

별점  리뷰내용
시장명                  
경동시장        125   125
광장시장        129   129
남대문시장       139   139
동묘시장        131   131
망원시장        135   135
수유시장        155   155
청량리청과물도매시장  140   140